In [5]:
from binance_trader import BinanceTrader
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")

trader = BinanceTrader(API_KEY, SECRET_KEY, testnet=False)
idx = pd.IndexSlice

In [2]:
params = {
    "symbol": ["BTCUSDT", "ETHUSDT"],
    "interval": "1m",
    "start_dt": "2024-11-01",
    "end_dt": "2024-12-22",
}


df = trader.get_candlestick_data_range(**params)

final_params = params.copy()
final_params["start_dt"] = df.index.get_level_values("Open Time").min()
final_params["end_dt"] = df.index.get_level_values("Open Time").max()
final_params["symbol"] = list(set(df.index.get_level_values("Symbol")))

final_params

{'symbol': ['ETHUSDT', 'BTCUSDT'],
 'interval': '1m',
 'start_dt': Timestamp('2024-11-05 13:24:00'),
 'end_dt': Timestamp('2024-12-21 23:00:00')}

In [3]:
for lag in [1, 2, 3, 10, 15]:
    for symbol in params["symbol"]:
        df.loc[idx[:, symbol], f"Target_{lag}"] = (
            df.loc[idx[:, symbol], "Close"].pct_change(lag).shift(-lag)
        )
        df.loc[idx[:, symbol], f"Return_{lag}"] = df.loc[
            idx[:, symbol], "Close"
        ].pct_change(lag)

df.dropna(inplace=True)
df.head(26)

Open      High       Low     Close    Volume  \
Open Time           Symbol                                                      
2024-11-05 13:39:00 BTCUSDT  68803.98  68813.90  68800.01  68800.84   0.06777   
                    ETHUSDT   2443.79   2448.27   2443.79   2444.13  16.30110   
2024-11-05 13:40:00 BTCUSDT  68802.29  69020.00  68736.02  68758.01   0.69902   
                    ETHUSDT   2444.31   2444.31   2443.63   2443.63  13.48000   
2024-11-05 13:41:00 BTCUSDT  68762.00  69020.00  68741.10  68758.72   0.26702   
                    ETHUSDT   2443.63   2443.63   2441.02   2442.03   7.78650   
2024-11-05 13:42:00 BTCUSDT  68753.93  68780.00  68738.20  68780.00   0.18968   
                    ETHUSDT   2441.98   2442.62   2441.22   2442.62   3.54840   
2024-11-05 13:43:00 BTCUSDT  68780.02  68785.88  68770.01  68772.00   0.08639   
                    ETHUSDT   2442.66   2442.70   2442.31   2442.53   2.40790   
2024-11-05 13:44:00 BTCUSDT  68772.00  68805.56  14814.00  68792.99   2.06546   
                    ETHUSDT   2442.63   2447.74   2442.63   2443.72   5.30170   
2024-11-05 13:45:00 BTCUSDT  68785.51  68786.23  68727.15  68727.15   0.16174   
                    ETHUSDT   2443.73   2443.88   2441.18   2441.18   4.43370   
2024-11-05 13:46:00 BTCUSDT  68725.96  68739.88  68721.29  68728.01   0.19609   
                    ETHUSDT   2441.01   2441.74   2440.82   2440.82   1.98450   
2024-11-05 13:47:00 BTCUSDT  68728.01  68797.56  68727.98  68792.72   0.25354   
                    ETHUSDT   2440.82   2445.67   2440.69   2443.63   4.94710   
2024-11-05 13:48:00 BTCUSDT  68776.01  68807.00  68775.50  68807.00   0.34538   
                    ETHUSDT   2443.09   2444.61   2442.61   2444.61   5.06640   
2024-11-05 13:49:00 BTCUSDT  68807.00  68833.67  68799.31  68833.67   0.29877   
                    ETHUSDT   2444.64   2445.87   2444.64   2445.64   4.26430   
2024-11-05 13:50:00 BTCUSDT  68835.70  68835.70  68814.01  68824.25   0.08711   
                    ETHUSDT   2445.23   2445.24   2444.65   2444.80   2.18410   
2024-11-05 13:51:00 BTCUSDT  68825.09  68825.41  68770.01  68785.20   0.24180   
                    ETHUSDT   2444.30   2444.30   2442.01   2442.29   3.85920   

                             Quote Asset Volume  Number of Trades  \
Open Time           Symbol                                          
2024-11-05 13:39:00 BTCUSDT         4662.813659              21.0   
                    ETHUSDT        39838.246804             110.0   
2024-11-05 13:40:00 BTCUSDT        48122.660763             105.0   
                    ETHUSDT        32940.695064              84.0   
2024-11-05 13:41:00 BTCUSDT        18364.161867              75.0   
                    ETHUSDT        19019.995090              72.0   
2024-11-05 13:42:00 BTCUSDT        13041.774572              64.0   
                    ETHUSDT         8665.206090              42.0   
2024-11-05 13:43:00 BTCUSDT         5941.595113              29.0   
                    ETHUSDT         5881.337170              28.0   
2024-11-05 13:44:00 BTCUSDT        53375.499429             133.0   
                    ETHUSDT        12955.546878              62.0   
2024-11-05 13:45:00 BTCUSDT        11120.899177              54.0   
                    ETHUSDT        10829.671160              48.0   
2024-11-05 13:46:00 BTCUSDT        13477.070150              55.0   
                    ETHUSDT         4844.752030              22.0   
2024-11-05 13:47:00 BTCUSDT        17430.781471              79.0   
                    ETHUSDT        12080.425706              59.0   
2024-11-05 13:48:00 BTCUSDT        23760.374447              85.0   
                    ETHUSDT        12380.973692              60.0   
2024-11-05 13:49:00 BTCUSDT        20559.527369              71.0   
                    ETHUSDT        10426.960257              46.0   
2024-11-05 13:50:00 BTCUSDT         5995.170702              26.0   
                    ETHUSDT         533

In [6]:
trader.get_position("BTCUSDT")

[2024-12-30 22:33:22,879] ERROR - Erreur lors de la récupération de la position: APIError(code=-2015): Invalid API-key, IP, or permissions for action.
